In [1]:
import torch
dev=torch.device('cpu')
centroids=torch.load('/home/jzgrp/xiaomenghan/vptq_qwen2-7/vptq_qwen2-7/model.layers.0.self_attn.q_proj_centroids.pt').to(dev)
index=torch.load('/home/jzgrp/xiaomenghan/vptq_qwen2-7/model.layers.0.self_attn.q_proj_indices.pt').to(dev)
print(index.shape)
centroids=centroids.weight.view(1,65536,8)
# high_mask=0xffff0000
# low_mask=0x0000ffff
# new_index=torch.zeros([448,3584]).to(dev).to(torch.int32).to(dev)
# for j in range(index.shape[1]):
#     for k in range(index.shape[2]):
#         packint32=index[0,j,k].to(torch.int32)
#         new_index[j,2*k+1]=torch.bitwise_right_shift(packint32.to(torch.int64),16).to(torch.uint16)
#         new_index[j,2*k]=torch.bitwise_and(packint32, 0xFFFF).to(torch.uint16)
#矢量化加速
# centroids = centroids.weight.view(1, 65536, 8)
high_mask = 0xffff0000  # 未使用，可保留或删除
low_mask = 0x0000ffff   # 未使用，可保留或删除
new_index = torch.zeros([448, 3584], dtype=torch.int32, device=dev)

# 矢量化操作
lower = index[0] & 0xFFFF              # 提取低16位，形状 (448, 1792)
upper = (index[0] >> 16) & 0xFFFF      # 提取高16位，形状 (448, 1792)
new_index = torch.stack([lower, upper], dim=2).view(448, 3584)  # 重组为 (448, 3584)


/tmp/ipykernel_1893859/2213283413.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  centroids=torch.load('/home/jzgrp/xiaomenghan/vptq_qwen2-7/layers0_q_centroids.pt').to(

torch.Size([1, 448, 1792])


/tmp/ipykernel_1893859/2213283413.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  index=torch.load('/home/jzgrp/xiaomenghan/vptq_qwen2-7/model.layers.0.self_attn.q_proj_

In [4]:
centroids = centroids.view(65536, 8).to(dev)  # 修正拼写错误并准备张量
recon_weight = torch.zeros([448, 3584], device=dev)
assert new_index.min() >= 0 and new_index.max() < 65536, "new_index 包含越界索引"
# 使用矢量化操作替代嵌套循环
recon_weight = centroids[new_index, 0]

print(recon_weight)

tensor([[-0.0425,  0.0938,  0.3320,  ...,  0.3301, -1.1328,  0.2773],
        [-1.2422,  0.0864, -0.7734,  ..., -0.7617,  0.5703,  0.6680],
        [-0.5898, -0.5234,  0.9375,  ...,  0.5547,  0.1206, -1.3125],
        ...,
        [-0.3516, -0.2197, -0.7109,  ..., -0.3730, -0.7539, -1.0156],
        [ 0.0184,  0.0171,  0.1187,  ...,  0.6602, -0.2695, -1.1719],
        [ 0.0171, -0.4277, -0.8867,  ...,  1.0156,  0.2383, -0.5039]],
       grad_fn=<IndexBackward0>)


In [3]:

centroids = centroids.view(16384, 32).to(device=dev, dtype=torch.float32)
# 创建 code 张量，确保在 dev 上并为 float32
code = torch.zeros([16384, 4], dtype=torch.float32, device=dev)
code = centroids[:, [0, 8, 16, 24]]
print(code)

tensor([[-0.3047,  0.2969,  0.0869, -0.6758],
        [-1.5859, -0.3848, -0.5078,  0.6797],
        [ 0.3398,  0.5508,  0.2969,  0.7734],
        ...,
        [-0.6758,  1.3359,  1.9375,  0.6289],
        [-0.5391, -1.8281, -0.0403,  0.1484],
        [-1.6719,  1.6875,  0.3750, -1.1406]], grad_fn=<IndexBackward0>)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
yx=new_index.cpu().detach()
y=yx[0,0:50]
x=range(50)
plt.figure(figsize=(20, 12))  # 设置图形大小
plt.scatter(x, y, label='y = sin(x)', color='b')  # 绘制折线图

# 添加标题和标签
plt.title('1D Data Plot', fontsize=16)
plt.xlabel('X-axis', fontsize=30)
plt.ylabel('Y-axis', fontsize=12)

# 添加图例
plt.legend()

# 添加网格线
plt.grid(True)

# 显示图形
plt.show()

In [ ]:
def cache_hit(index=torch.Tensor,cache_size=None,dev=None):
    index_pool=torch.zeros([cache_size],dtype=torch.int32).cpu()
    allindex=index.shape[0]*index.shape[1]
    hit=0
    a=int(cache_size/2-1)
    index=index.cpu()
    for i in range(index.shape[0]):
        for j in range(index.shape[1]):
            if torch.eq(index_pool, index[i][j]).any() :
                hit=hit+1
            else:
                if index[i][j]<=a:
                    index_pool=torch.arange(0,cache_size)
                else:
                    index_pool[0:a]=torch.arange(index[i][j]-a,index[i][j])
                    index_pool[a+1:cache_size-1]=torch.arange(index[i][j],index[i][j]+a)#缺少一个右边界条件

    return hit,hit/allindex
print(cache_hit(new_index,500,dev))

In [ ]:
import torch
a=torch.Tensor([[1,0,1],
                [1,1,1]])
a=a.t()
print(torch.nonzero(a!=1)[0][0])


In [ ]:
import torch
class cache_sim:
    def __init__(self, mode, linesize, cachesize, num_lin, codebook):
        self.mode = mode
        self.linesize = linesize  # 这里的linesize表示能装几个code
        self.cachesize = cachesize
        self.num_lin = num_lin
        # self.uesd_record = torch.ones([self.num_lin]) * -1  # 第一列表示是否为空，第二列表示最近使用时间.
        self.cache = torch.ones([self.num_lin, self.linesize]) * -1
        self.time = 0
        self.codebook = codebook
        self.l = []  # 记录使用

    def maintain_record(self, used_line_num):
        # print(used_line_num)
        if used_line_num in self.l:
            self.l.remove(used_line_num) # 如果出现在self l中 那么都会被删除 然后将其补至最后  确保LRU替换策略
            self.l.append(used_line_num)
        else:
            self.l.append(used_line_num)

    def is_empty(self):
        if len(self.l) < self.num_lin:
            return -1
        else:
            return self.l.pop(0)  # 弹出最早使用的序号

    def load_data(self, data):
        indices = self.is_empty()
        # print(self.codebook,data)
        in_code = torch.nonzero(self.codebook == data)[0, 0].item()
        # print(indices)
        if indices != -1:
            # print(indices)
            # print(self.cache.shape,self.codebook.shape)
            self.cache[indices] = self.codebook[in_code]
            self.maintain_record(indices)
        else:
            for x in range(self.num_lin):
                if x in self.l:
                    continue
                else:
                    self.cache[x] = self.codebook[in_code]
                    self.maintain_record(x)
                    break

    def sim(self, read_data):
        indices = torch.nonzero(self.cache == read_data) 
        if torch.numel(indices) == 0:
            indices=indices
            self.load_data(read_data)
            return False
        else:
            indices = indices[0]
            self.maintain_record(indices[0])
            return True

cache = cache_sim(mode=None, linesize=4, cachesize=None, num_lin=256, codebook=code)
sum_cnt=0
hit_cnt=0
for i in range(448):
        for k in range(3584):
            sum_cnt=sum_cnt+1
            if cache.sim(recon_weight[i,k])==True:
                hit_cnt=hit_cnt+1
            else:
                continue


In [36]:
import torch
from collections import OrderedDict, defaultdict

class cache_sim:
    def __init__(self, mode, linesize, cachesize, num_lin, codebook):
        self.linesize = linesize
        self.num_lin = num_lin
        self.cache = torch.full((num_lin, linesize), -1)
        self.codebook = codebook
        
        # 预建 codebook 值到行的快速映射 (值 -> 行索引)
        self.codebook_value_map = {}
        for row_idx, row in enumerate(codebook):
            for val in row:
                val = val.item()
                if val not in self.codebook_value_map:
                    self.codebook_value_map[val] = row_idx
        
        # 使用 OrderedDict 实现高效 LRU 机制
        self.lru = OrderedDict()
        
        # 值到缓存行的反向映射 (值 -> 所在行集合)
        self.value_to_lines = defaultdict(set)
        # 缓存行到值的反向映射 (行 -> 存储的值集合)
        self.line_to_values = [set() for _ in range(num_lin)]

    def maintain_record(self, line):
        """ 更新 LRU 记录，O(1) 时间复杂度 """
        if line in self.lru:
            self.lru.move_to_end(line)
        else:
            self.lru[line] = None

    def is_empty(self):
        """ 返回可替换的行或 -1 """
        if len(self.lru) < self.num_lin:
            return -1
        else:
            # 弹出 LRU 行并清理其反向映射
            lru_line, _ = self.lru.popitem(last=False)
            for val in self.line_to_values[lru_line]:
                self.value_to_lines[val].remove(lru_line)
                if not self.value_to_lines[val]:
                    del self.value_to_lines[val]
            self.line_to_values[lru_line].clear()
            return lru_line

    def load_data(self, data):
        """ 加载数据到缓存，向量化操作 """
        data_val = data.item()
        if data_val not in self.codebook_value_map:
            raise ValueError(f"Value {data_val} not in codebook")
        
        code_row = self.codebook[self.codebook_value_map[data_val]]
        target_line = self.is_empty()
        
        if target_line == -1:
            # 寻找未使用的行
            for line in range(self.num_lin):
                if line not in self.lru:
                    target_line = line
                    break
            # 如果所有行已使用，触发 LRU 替换
            if target_line == -1:
                target_line = self.is_empty()
        # print(target_line)
        # 更新缓存行数据
        self.cache[target_line] = code_row
        # 更新反向映射
        new_values = set(code_row.tolist())
        for val in new_values:
            self.value_to_lines[val].add(target_line)
        self.line_to_values[target_line] = new_values
        self.maintain_record(target_line)

    def sim(self, read_data):
        """ 批量查询优化，O(1) 时间复杂度 """
        data_val = read_data.item()
        
        # 
        if data_val in self.value_to_lines:
            # 取第一个关联行并更新 LRU
            line = next(iter(self.value_to_lines[data_val]))
            self.maintain_record(line)
            return True
        else:
            self.load_data(read_data)
            # print('not hit')
            return False


# 测试循环优化
def run_simulation(cache, recon_weight):
    # 将输入数据转换为向量化操作
    recon_flatten = recon_weight.view(-1)
    hit_cnt = torch.sum(torch.tensor([
        cache.sim(val) for val in recon_flatten
    ], dtype=torch.int32))
    return hit_cnt.item(), len(recon_flatten)

# 初始化
cache = cache_sim(mode=None, linesize=4, cachesize=None, num_lin=256, codebook=code)
hit_cnt, sum_cnt = run_simulation(cache, recon_weight)
print(f"Hit Rate: {hit_cnt / sum_cnt:.4f}")

Hit Rate: 0.6140


In [31]:
import torch
from collections import deque, defaultdict

class cache_sim_fifo:
    def __init__(self, mode, linesize, cachesize, num_lin, codebook):
        self.linesize = linesize
        self.num_lin = num_lin
        self.cache = torch.full((num_lin, linesize), -1)
        self.codebook = codebook
        
        # 预建 codebook 值到行的快速映射 (值 -> 行索引)
        self.codebook_value_map = {}
        for row_idx, row in enumerate(codebook):
            for val in row:
                val = val.item()
                if val not in self.codebook_value_map:
                    self.codebook_value_map[val] = row_idx
        
        # FIFO 队列 (核心改动)
        self.fifo_queue = deque(maxlen=num_lin)  # 自动淘汰最早进入的项
        
        # 值到缓存行的反向映射 (值 -> 所在行集合)
        self.value_to_lines = defaultdict(set)
        # 缓存行到值的反向映射 (行 -> 存储的值集合)
        self.line_to_values = [set() for _ in range(num_lin)]

    def maintain_record(self, line):
        """ FIFO 只在加载时记录新行，不调整顺序 (核心改动) """
        if line not in self.fifo_queue:
            self.fifo_queue.append(line)

    def is_empty(self):
        """ 返回可替换的行或 -1 """
        if len(self.fifo_queue) < self.num_lin:
            return -1
        else:
            # 弹出 FIFO 队列最旧的行 (核心改动)
            lru_line = self.fifo_queue.popleft()
            # 清理反向映射
            for val in self.line_to_values[lru_line]:
                self.value_to_lines[val].remove(lru_line)
                if not self.value_to_lines[val]:
                    del self.value_to_lines[val]
            self.line_to_values[lru_line].clear()
            return lru_line

    def load_data(self, data):
        """ 加载数据到缓存 """
        data_val = data.item()
        if data_val not in self.codebook_value_map:
            raise ValueError(f"Value {data_val} not in codebook")
        
        code_row = self.codebook[self.codebook_value_map[data_val]]
        target_line = self.is_empty()
        
        if target_line == -1:
            # 寻找未使用的行 (新逻辑)
            for line in range(self.num_lin):
                if line not in self.fifo_queue:
                    target_line = line
                    break
            # 如果所有行已使用，触发 FIFO 替换
            if target_line == -1:
                target_line = self.is_empty()

        # 更新缓存行数据
        self.cache[target_line] = code_row
        # 更新反向映射
        new_values = set(code_row.tolist())
        for val in new_values:
            self.value_to_lines[val].add(target_line)
        self.line_to_values[target_line] = new_values
        self.maintain_record(target_line)  # 仅在此处更新队列

    def sim(self, read_data):
        """ 查询优化 (命中时不更新队列顺序) """
        data_val = read_data.item()
        if data_val in self.value_to_lines:
            # FIFO 不更新访问顺序 (核心改动)
            return True
        else:
            self.load_data(read_data)
            return False

# 使用示例
cache_fifo = cache_sim_fifo(
    mode=None, linesize=4, cachesize=None, 
    num_lin=256, codebook=code
)

# 测试方法与之前相同
def run_simulation(cache, recon_weight):
    recon_flatten = recon_weight.view(-1)
    hit_cnt = torch.sum(torch.tensor([
        cache.sim(val) for val in recon_flatten
    ], dtype=torch.int32))
    return hit_cnt.item(), len(recon_flatten)

hit_cnt, sum_cnt = run_simulation(cache_fifo, recon_weight)
print(f"FIFO Hit Rate: {hit_cnt / sum_cnt:.4f}")

FIFO Hit Rate: 0.5980


In [ ]:
class cache_sim_set_associative_fifo:
    def __init__(self, mode, linesize, cachesize, num_lin, codebook, associativity=4):
        # 参数校验
        assert num_lin % associativity == 0, "Cache lines must be divisible by associativity"
        assert associativity >= 1, "Associativity must be at least 1"
        
        # 基础参数
        self.linesize = linesize
        self.num_lin = num_lin
        self.associativity = associativity
        self.groups = num_lin // associativity  # 计算组数
        self.cache = torch.full((num_lin, linesize), -1)
        self.codebook = codebook

        # 预建codebook值到行的映射
        self.codebook_value_map = {}
        for row_idx, row in enumerate(codebook):
            for val in row:
                val = val.item()
                if val not in self.codebook_value_map:
                    self.codebook_value_map[val] = row_idx

        # 为每个组维护FIFO队列
        self.group_queues = [deque(maxlen=associativity) for _ in range(self.groups)]

        # 反向映射数据结构
        self.value_to_lines = defaultdict(set)
        self.line_to_values = [set() for _ in range(num_lin)]

    def _get_replacement_line(self, group):
        """ 在指定组中获取需要替换的缓存行 """
        # 计算组内起始行号
        start = group * self.associativity
        # 优先寻找空闲行
        for line_offset in range(self.associativity):
            current_line = start + line_offset
            if current_line not in self.group_queues[group]:
                return current_line

        # 执行FIFO替换
        victim = self.group_queues[group].popleft()
        
        # 清理反向映射
        for val in self.line_to_values[victim]:
            self.value_to_lines[val].remove(victim)
            if not self.value_to_lines[val]:
                del self.value_to_lines[val]
        self.line_to_values[victim].clear()
        
        return victim

    def load_data(self, data):
        """ 加载数据到缓存 """
        data_val = data.item()
        if data_val not in self.codebook_value_map:
            raise ValueError(f"Value {data_val} not in codebook")

        # 获取对应的codebook行和组号
        code_row_idx = self.codebook_value_map[data_val]
        code_row = self.codebook[code_row_idx]
        group = code_row_idx % self.groups  # 关键组映射逻辑

        # 获取目标缓存行
        target_line = self._get_replacement_line(group)

        # 更新缓存内容
        self.cache[target_line] = code_row

        # 维护组队列
        self.group_queues[group].append(target_line)

        # 更新反向映射
        new_values = set(code_row.tolist())
        self.line_to_values[target_line] = new_values
        for val in new_values:
            self.value_to_lines[val].add(target_line)

    def sim(self, read_data):
        """ 模拟缓存访问 """
        data_val = read_data.item()
        if data_val in self.value_to_lines:
            return True  # 命中不更新队列
        else:
            self.load_data(read_data)
            return False
        
cache_fifo = cache_sim_set_associative_fifo(
    mode=None, linesize=4, cachesize=None, 
    num_lin=256, codebook=code,
    associativity=8
)

# 测试方法与之前相同
def run_simulation(cache, recon_weight):
    recon_flatten = recon_weight.view(-1)
    hit_cnt = torch.sum(torch.tensor([
        cache.sim(val) for val in recon_flatten
    ], dtype=torch.int32))
    return hit_cnt.item(), len(recon_flatten)

hit_cnt, sum_cnt = run_simulation(cache_fifo, recon_weight)
print(f"FIFO Hit Rate: {hit_cnt / sum_cnt:.4f}")